In [1]:
!unzip Data.zip

Archive:  Data.zip
   creating: Data/
  inflating: Data/.~lock.ballbyball.csv#  
  inflating: Data/ballbyball.csv     
  inflating: Data/country.csv        
  inflating: Data/ground.csv         
  inflating: Data/matches.csv        
  inflating: Data/players.csv        
  inflating: Data/season.csv         
  inflating: Data/team.csv           
  inflating: Data/town.csv           


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import SMOTE
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [3]:
ball_df = pd.read_csv('Data/ballbyball.csv')
country_df = pd.read_csv('Data/country.csv')
ground_df = pd.read_csv('Data/ground.csv')
matches_df = pd.read_csv('Data/matches.csv')
players_df = pd.read_csv('Data/players.csv')
season_df = pd.read_csv('Data/season.csv')
team_df = pd.read_csv('Data/team.csv')
town_df = pd.read_csv('Data/town.csv')

In [4]:
ball_df['totalRuns'] = ball_df['totalRuns'] - ball_df['run']
ball_df['totalWickets'] = ball_df['totalWickets'] - ball_df['isWicket'].apply(lambda x: 1 if x else 0)

In [5]:
ground_df.drop(['Ground Slug', 'Town ID', 'Capacity'], axis=1, inplace=True)

In [6]:
matches_df.rename(columns={'ground_id': 'Ground ID'}, inplace=True)

matches_df = pd.merge(matches_df, ground_df, on='Ground ID', how='inner')

In [7]:
matches_df.drop(['season_id', 'slug', 'title', 'date', 'time', 'status', 'status_description', 'winner_team_id', 'toss_winner_team_id', 'toss_choice', 'Ground ID', 'team_1_id', 'team_2_id', 'team_1_score', 'team_1_scoreInfo', 'team_2_scoreInfo'], axis=1, inplace=True)

In [8]:
players_df.drop(['Full Name', 'Image', 'DOB', 'Batting Style (s)', 'Bowling Style (s)', 'Team ID'], axis=1, inplace=True)

In [9]:
ball_df = pd.merge(ball_df, matches_df, on='match_id', how='inner')

In [10]:
ball_df.drop(['index', 'overNumber', 'ballNumber', 'oversUnique', 'penalties', 'batsmanRuns', 'outPlayerId'], axis=1, inplace=True)

In [11]:
ball_df = ball_df.merge(players_df, how='left', left_on='batsmanPlayerId', right_on='ID', suffixes=('', '_batsman'))

ball_df.rename(columns={
    'Name': 'Batsman_Name',
    'Role': 'Batsman_Role',
    'Batting Style (l)': 'Batsman_Batting_Style',
    'Bowling Style(l)': 'Batsman_Bowling_Style',
    'Playing Role': 'Batsman_Playing_Role'
}, inplace=True)

ball_df.drop(columns=['ID'], inplace=True)

ball_df = ball_df.merge(players_df, how='left', left_on='bowlerPlayerId', right_on='ID', suffixes=('', '_bowler'))

ball_df.rename(columns={
    'Name': 'Bowler_Name',
    'Role': 'Bowler_Role',
    'Batting Style (l)': 'Bowler_Batting_Style',
    'Bowling Style(l)': 'Bowler_Bowling_Style',
    'Playing Role': 'Bowler_Playing_Role'
}, inplace=True)

ball_df.drop(columns=['ID'], inplace=True)

In [12]:
ball_df.drop(columns=['batsmanPlayerId', 'bowlerPlayerId'], inplace=True)

In [13]:
ball_df.drop(columns=['Batsman_Bowling_Style', 'Bowler_Batting_Style'], inplace=True)

In [14]:
df = ball_df.dropna(subset=['pitchLength', 'pitchLine','shotType','Batsman_Name','Batsman_Batting_Style', 'Batsman_Playing_Role','Bowler_Name','Bowler_Bowling_Style','Bowler_Playing_Role'])

In [15]:
df.drop(columns=['match_id'], inplace=True)

<ipython-input-15-25476660b3a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['match_id'], inplace=True)


In [16]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
X = df.drop('isWicket', axis=1)
y = df['isWicket']

categorical_cols = X.select_dtypes(include='object').columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

columns_to_scale = [col for col in X.columns if col not in ['pitchLine', 'pitchLength', 'Bowler_Bowling_Style', 'shotType']]

scaler = StandardScaler()
X_resampled[columns_to_scale] = scaler.fit_transform(X_resampled[columns_to_scale])

df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['isWicket'] = y_resampled

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     tf.config.set_logical_device_configuration(physical_devices[0],
#                                                [tf.config.LogicalDeviceConfiguration(memory_limit=8192)])
#     tf.config.set_visible_devices(physical_devices[0], 'GPU')

input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets',
              'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role',
              'Batsman_Batting_Style', 'Batsman_Playing_Role']
target_outputs = ['shotType']

X_all = pd.get_dummies(df_resampled[input_cols])

for target in target_outputs:
    print(f"\n------ Training model for {target} ------")

    y_all = df_resampled[target]
    y_all_encoded, class_names = pd.factorize(y_all)

    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all_encoded, test_size=0.2, random_state=42, stratify=y_all_encoded
    )

    num_classes = len(class_names)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test_labels = y_test.copy()
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    with tf.device('/GPU:0'):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)

        print(f"\n--- Results for {target} ---")
        print("Accuracy:", accuracy_score(y_test_labels, y_pred))
        print(classification_report(y_test_labels, y_pred, target_names=[str(cls) for cls in class_names]))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        ax1.plot(history.history['accuracy'], label='Train Acc')
        ax1.plot(history.history['val_accuracy'], label='Val Acc')
        ax1.set_title('Accuracy over Epochs')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()

        ax2.plot(history.history['loss'], label='Train Loss')
        ax2.plot(history.history['val_loss'], label='Val Loss')
        ax2.set_title('Loss over Epochs')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()

        plt.tight_layout()
        plot_filename = f'{target}_training_plot.png'
        plt.savefig(plot_filename)
        print(f"Training plots saved to {plot_filename}")
        plt.close()

        model_dir = f'{target}_model.h5'
        model.save(model_dir)
        print(f"Model saved to directory: {model_dir}")


------ Training model for shotType ------


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1107 - loss: 2.9149 - val_accuracy: 0.1347 - val_loss: 2.8214
Epoch 2/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1428 - loss: 2.8019 - val_accuracy: 0.1382 - val_loss: 2.7909
Epoch 3/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1471 - loss: 2.7829 - val_accuracy: 0.1461 - val_loss: 2.7874
Epoch 4/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1499 - loss: 2.7743 - val_accuracy: 0.1484 - val_loss: 2.7731
Epoch 5/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1535 - loss: 2.7628 - val_accuracy: 0.1514 - val_loss: 2.7687
Epoch 6/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1607 - loss: 2.7420 - val_accuracy: 0.1575 - val_loss: 2.7538
Epoch 7/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1650 - loss: 2.7261 - val_accuracy: 0.1531 - val_loss: 2.7549
Epoch 8/50
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1629 - loss: 2.7185 -

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training plots saved to shotType_training_plot.png
Model saved to directory: shotType_model.h5


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
threshold = 2
df_copy = df_resampled.copy()
df_copy['target'] = ((df_copy['isWicket'] == 1) | (df_copy['run'] <= threshold)).astype(int)

input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets',
              'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role',
              'Batsman_Batting_Style', 'Batsman_Playing_Role']
target_outputs = ['pitchLine', 'pitchLength', 'Bowler_Bowling_Style']

df_good = df_copy[df_copy['target'] == 1]
X_all = pd.get_dummies(df_good[input_cols])

for target in target_outputs:
    print(f"\n------ Training model for {target} ------")

    y_all = df_good[target]
    y_all_encoded, class_names = pd.factorize(y_all)

    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all_encoded, test_size=0.2, random_state=42, stratify=y_all_encoded
    )

    num_classes = len(class_names)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test_labels = y_test.copy()
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    with tf.device('/GPU:0'):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)

        print(f"\n--- Results for {target} ---")
        print("Accuracy:", accuracy_score(y_test_labels, y_pred))
        print(classification_report(y_test_labels, y_pred, target_names=[str(cls) for cls in class_names]))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        ax1.plot(history.history['accuracy'], label='Train Accuracy')
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_title('Model Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()

        ax2.plot(history.history['loss'], label='Train Loss')
        ax2.plot(history.history['val_loss'], label='Validation Loss')
        ax2.set_title('Model Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()

        plt.tight_layout()
        plot_filename = f'{target}_training_plot.png'
        plt.savefig(plot_filename)
        print(f"Training plot saved to {plot_filename}")
        plt.close()

        model_dir = f'{target}_model_safe.h5'
        model.save(model_dir)
        print(f"Model saved to directory: {model_dir}")


------ Training model for pitchLine ------
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


959/959 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.4796 - loss: 1.1348 - val_accuracy: 0.5085 - val_loss: 1.0779
Epoch 2/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5085 - loss: 1.0658 - val_accuracy: 0.5075 - val_loss: 1.0596
Epoch 3/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5175 - loss: 1.0494 - val_accuracy: 0.5058 - val_loss: 1.0516
Epoch 4/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5251 - loss: 1.0354 - val_accuracy: 0.5156 - val_loss: 1.0411
Epoch 5/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5244 - loss: 1.0290 - val_accuracy: 0.5178 - val_loss: 1.0373
Epoch 6/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5325 - loss: 1.0132 - val_accuracy: 0.5237 - val_loss: 1.0268
Epoch 7/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5401 - loss: 1.0064 - val_accuracy: 0.5232 - val_loss: 1.0285
Epoch 8/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5514 - loss: 0.9874 - val_accuracy: 0.5352 - val_

Training plot saved to pitchLine_training_plot.png
Model saved to directory: pitchLine_model_safe.h5

------ Training model for pitchLength ------
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.4247 - loss: 1.5201 - val_accuracy: 0.4219 - val_loss: 1.4865
Epoch 2/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4360 - loss: 1.4596 - val_accuracy: 0.4270 - val_loss: 1.4675
Epoch 3/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4351 - loss: 1.4461 - val_accuracy: 0.4275 - val_loss: 1.4644
Epoch 4/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4407 - loss: 1.4303 - val_accuracy: 0.4273 - val_loss: 1.4505
Epoch 5/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4486 - loss: 1.4106 - val_accuracy: 0.4312 - val_loss: 1.4472
Epoch 6/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4429 - loss: 1.4099 - val_accuracy: 0.4338 - val_loss: 1.4387
Epoch 7/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.4526 - loss: 1.3934 - val_accuracy: 0.4397 - val_loss: 1.4301
Epoch 8/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4498 - loss: 1.3916 - val_accuracy: 0.4356 - val_

Training plot saved to pitchLength_training_plot.png
Model saved to directory: pitchLength_model_safe.h5

------ Training model for Bowler_Bowling_Style ------
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.1259 - loss: 2.7484 - val_accuracy: 0.1654 - val_loss: 2.6130
Epoch 2/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1768 - loss: 2.5993 - val_accuracy: 0.1825 - val_loss: 2.5762
Epoch 3/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1941 - loss: 2.5574 - val_accuracy: 0.1925 - val_loss: 2.5557
Epoch 4/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2047 - loss: 2.5273 - val_accuracy: 0.1984 - val_loss: 2.5360
Epoch 5/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2064 - loss: 2.5132 - val_accuracy: 0.2065 - val_loss: 2.5104
Epoch 6/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2236 - loss: 2.4673 - val_accuracy: 0.2123 - val_loss: 2.4979
Epoch 7/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2285 - loss: 2.4559 - val_accuracy: 0.2105 - val_loss: 2.4896
Epoch 8/50
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.2340 - loss: 2.4365 - val_accuracy: 0.2215 - val_

Training plot saved to Bowler_Bowling_Style_training_plot.png
Model saved to directory: Bowler_Bowling_Style_model_safe.h5


In [27]:
threshold = 2
df_copy = df_resampled.copy()
df_copy['target'] = ((df_copy['isWicket'] == 0) & (df_copy['run'] > threshold)).astype(int)

input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets',
              'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role',
              'Batsman_Batting_Style', 'Batsman_Playing_Role']
target_outputs = ['pitchLine', 'pitchLength', 'Bowler_Bowling_Style']

df_good = df_copy[df_copy['target'] == 1]
X_all = pd.get_dummies(df_good[input_cols])

for target in target_outputs:
    print(f"\n------ Training model for {target} ------")

    y_all = df_good[target]

    class_counts = y_all.value_counts()
    print(f"Class distribution for {target}:")
    print(class_counts)

    valid_classes = class_counts[class_counts >= 2].index.tolist()
    print(f"Valid classes with at least 2 samples: {len(valid_classes)}")

    df_filtered = df_good[df_good[target].isin(valid_classes)]
    X_filtered = pd.get_dummies(df_filtered[input_cols])
    y_filtered = df_filtered[target]

    y_filtered_encoded, class_names = pd.factorize(y_filtered)

    X_train, X_test, y_train, y_test = train_test_split(
        X_filtered, y_filtered_encoded, test_size=0.2, random_state=42, stratify=y_filtered_encoded
    )

    num_classes = len(class_names)
    print(f"Number of classes after filtering: {num_classes}")

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test_labels = y_test.copy()
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    with tf.device('/GPU:0'):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

        y_pred_probs = model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)

        print(f"\n--- Results for {target} ---")
        print("Accuracy:", accuracy_score(y_test_labels, y_pred))
        print(classification_report(y_test_labels, y_pred, target_names=[str(cls) for cls in class_names]))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        ax1.plot(history.history['accuracy'], label='Train Accuracy')
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_title('Model Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()

        ax2.plot(history.history['loss'], label='Train Loss')
        ax2.plot(history.history['val_loss'], label='Validation Loss')
        ax2.set_title('Model Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()

        plt.tight_layout()
        plot_filename = f'{target}_training_plot_filtered.png'
        plt.savefig(plot_filename)
        print(f"Training plot saved to {plot_filename}")
        plt.close()

        model_dir = f'{target}_model_filtered.h5'
        model.save(model_dir)
        print(f"Model saved to directory: {model_dir}")


------ Training model for pitchLine ------
Class distribution for pitchLine:
pitchLine
2    1881
1    1215
0     305
4     299
3      13
Name: count, dtype: int64
Valid classes with at least 2 samples: 5
Number of classes after filtering: 5
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.3016 - loss: 1.3877 - val_accuracy: 0.4848 - val_loss: 1.1422
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5179 - loss: 1.1317 - val_accuracy: 0.4848 - val_loss: 1.1400
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5064 - loss: 1.1118 - val_accuracy: 0.4882 - val_loss: 1.1280
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5372 - loss: 1.0989 - val_accuracy: 0.4916 - val_loss: 1.1307
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5053 - loss: 1.0859 - val_accuracy: 0.4949 - val_loss: 1.1284
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5171 - loss: 1.0883 - val_accuracy: 0.4933 - val_loss: 1.1378
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5246 - loss: 1.0859 - val_accuracy: 0.5000 - val_loss: 1.1414
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5268 - loss: 1.0761 - val_accuracy: 0.5000 - val_loss: 1.1428
Ep

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training plot saved to pitchLine_training_plot_filtered.png
Model saved to directory: pitchLine_model_filtered.h5

------ Training model for pitchLength ------
Class distribution for pitchLength:
pitchLength
2    1302
0     943
4     814
3     393
1     233
5      28
Name: count, dtype: int64
Valid classes with at least 2 samples: 6
Number of classes after filtering: 6
Epoch 1/50


...

75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.2497 - loss: 1.7366 - val_accuracy: 0.3687 - val_loss: 1.5539
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3402 - loss: 1.5166 - val_accuracy: 0.3653 - val_loss: 1.5101
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3448 - loss: 1.4862 - val_accuracy: 0.3552 - val_loss: 1.5020
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3341 - loss: 1.4681 - val_accuracy: 0.3485 - val_loss: 1.5089
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3355 - loss: 1.4639 - val_accuracy: 0.3636 - val_loss: 1.5000
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3644 - loss: 1.4752 - val_accuracy: 0.3670 - val_loss: 1.4980
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3548 - loss: 1.4571 - val_accuracy: 0.3653 - val_loss: 1.5023
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3518 - loss: 1.4397 - val_accuracy: 0.3434 - val_loss: 1.5035
Ep

Training plot saved to pitchLength_training_plot_filtered.png
Model saved to directory: pitchLength_model_filtered.h5

------ Training model for Bowler_Bowling_Style ------
Class distribution for Bowler_Bowling_Style:
Bowler_Bowling_Style
9     654
8     610
13    537
18    413
15    313
1     300
6     224
0     158
4     144
11    139
7     117
16     33
17     25
5      16
2      11
12     11
3       4
10      3
14      1
Name: count, dtype: int64
Valid classes with at least 2 samples: 18
Number of classes after filtering: 18
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0810 - loss: 2.8330

InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipy